# Julia

### DataFrames

Utilidade: Manipulação de dados.

In [1]:
using Pkg
Pkg.add("DataFrames")
using DataFrames

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %Fetching: [=====>                                   ]  11.3 %>                              ]  24.8 %>                       ]  41.5 %=======================>                 ]  55.1 %          ]  74.7 %  ]  94.8 %Fetching: [=======================================> ]  97.0 %Fetching: [========================================>]  97.6 %

  Resolving package versions...
  Installed MbedTLS_jll ──────── v2.16.8+1
  Installed DataAPI ──────────── v1.4.0
  Installed OffsetArrays ─────── v1.4.0
  Installed DataDeps ─────────── v0.7.3
  Installed NNlib ────────────── v0.7.6
  Installed StaticArrays ─────── v0.12.5
  Installed FileIO ───────────── v1.4.4
  Installed Documenter ───────── v0.25.3
  Installed IOCapture ────────── v0.1.1
  Installed ArrayInterface ───── v2.13.7
  Installed ChainRules ───────── v0.7.27
  Installed Polynomials ──────── v1.1.11
  Installed Conda ────────────── v1.5.0
  Installed MbedTLS ──────────── v1.0.3
  Installed TimeZones ────────── v1.5.0
  Installed HDF5_jll ─────────── v1.10.5+7
  Installed OpenBLAS_jll ─────── v0.3.9+5
  Installed WoodburyMatrices ─── v0.5.3
  Installed TimerOutputs ─────── v0.5.7
  Installed JLLWrappers ──────── v1.1.3
  Installed OrderedCollections ─ v1.3.2
######################################################################### 100,0%#=#=-#  #                          